In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#eda
#check distribution
#sheet 4 why different?
#change test rank sum test
#draw graph
#compute logistic
#maybe combine all 49*168 data together

In [ ]:
#data merging
sheet_num = [0,1,2,3,5]
sheet_rows = [13,5,6,12,9,4]
start_row = [1,14,19,25,37,46]
each_mut = 168
merge = pd.DataFrame()
for sheet in sheet_num:
    data_slice = []
    #讀資料
    data = pd.read_excel ('/Users/liu/Desktop/合作社專案三_lungcancer/data(new)/LoD+LoB(new heater) data整理-03102020.xlsx',sheet_name=sheet)
    rows = sheet_rows[sheet]
    start = start_row[sheet]
    #整理資料
    data = data.drop('Unnamed: 0',axis=1)
    data = data.dropna(axis=1,how='all')
    for i in range(rows):
        data_slice.append(data.iloc[start+i,:].dropna())
        x = pd.DataFrame(np.array(data_slice,dtype=object))
    merge = pd.concat([merge,x],axis=0)

In [ ]:
#data merging for sheet4
sheet4 = pd.read_csv('/Users/liu/Desktop/合作社專案三_lungcancer/data(new)/LoD_EGFR-2-03102020.csv')
rows = sheet_rows[4]
start = start_row[4]
mergefor4 = pd.DataFrame()
for i in range(rows):
        mergefor4 = pd.concat([mergefor4,pd.DataFrame(sheet4.iloc[start+i,:].dropna()).reset_index(drop=True).transpose()],axis=0)
merge = pd.concat([merge.iloc[0:36,:],mergefor4,merge.iloc[36:40,:]],axis=0).dropna(axis=1).reset_index(drop=True)

In [ ]:
#change their column name
column_name = {}
for i in range(4):
    for j in range(21):
        column_name.update({j+1+i*21: 'b1_'+'l'+str(4-i)+'r'+str(j)})
for i in range(4):
    for j in range(21):
        column_name.update({j+1+i*21+84: 'b2_'+'l'+str(4-i)+'r'+str(j)})
len(column_name)
merge = merge.rename(columns=column_name)
merge.iloc[:,1:169] = merge.iloc[:,1:169].astype('float')

In [ ]:
#demo the distribution of one mutation
cut = (np.linspace(0,168,9)+1).astype(int)
fig, axs = plt.subplots(2,4,figsize=(20,10))
for i in range(len(cut)-1):
    if i < 4:
        sns.histplot(ax=axs[0,i],x=merge.iloc[0,cut[i]:cut[i+1]]).set_title(label=merge.iloc[0,0]+'_'+str(7-i))
    else:
        sns.histplot(ax=axs[1,i-4],x=merge.iloc[0,cut[i]:cut[i+1]]).set_title(label=merge.iloc[0,0]+'_'+str(7-i))

In [ ]:
#drawing all
while true:
    pass
    for index in range(25):
    fig, axs = plt.subplots(2,4,figsize=(20,10))
    for i in range(len(cut)-1):
        if i < 4:
            sns.histplot(ax=axs[0,i],x=merge.iloc[index,cut[i]:cut[i+1]]).set_title(label=merge.iloc[index,0]+'_'+str(i))
        else:
            sns.histplot(ax=axs[1,i-4],x=merge.iloc[index,cut[i]:cut[i+1]]).set_title(label=merge.iloc[index,0]+'_'+str(i))

In [ ]:
#drawing all part2
while true:
    pass
    for index in range(25,49):
    fig, axs = plt.subplots(2,4,figsize=(20,10))
    for i in range(len(cut)-1):
        if i < 4:
            sns.histplot(ax=axs[0,i],x=merge.iloc[index,cut[i]:cut[i+1]]).set_title(label=merge.iloc[index,0]+'_'+str(i))
        else:

            sns.histplot(ax=axs[1,i-4],x=merge.iloc[index,cut[i]:cut[i+1]]).set_title(label=merge.iloc[index,0]+'_'+str(i))

In [ ]:
#rank sum test for 2 batch of products
from scipy.stats import ranksums
count = 0
alpha = 0.01
for i in range(merge.shape[0]):
    batch1 = merge.iloc[i,1:85]
    batch2 = merge.iloc[i,85:169]
    stats,pvalue = ranksums(batch1,batch2)
    print('The pvalue of index {}, {} is: {}'.format(i,merge.iloc[i,0],pvalue))
    if pvalue < alpha:
        print('***Significant***')
        count += 1
print('There are {} tests are significant'.format(count))
#alpha = 0.05 , 6 significant tests
#alpha = 0.01 , 4 significant tests

In [ ]:
#try to see whether the discrepency vary from different desity of mutation
#store a dataframe 49*4 of True/False
from scipy.stats import ranksums
count_2 = pd.DataFrame(index=range(49),columns=range(4))
alpha = 0.05
for i in range(merge.shape[0]):
    for j in range(4):
        start_1 = j*21+1
        start_2 = j*21+85
        batch1 = merge.iloc[i,start_1:start_1+21]
        batch2 = merge.iloc[i,start_2:start_2+21] 
        stats,pvalue = ranksums(batch1,batch2)
        print('The pvalue of index {}, {} is: {}'.format(i,merge.iloc[i,j],pvalue))
        if pvalue < alpha:
            print('***Significant***')
            count_2.iloc[i,j] = True
        else:
            count_2.iloc[i,j] = False

In [ ]:
#observe count_2
column_sum = np.sum(count_2,axis=1)
row_sum = np.sum(count_2,axis=0)

In [ ]:
column_sum[column_sum>=3]

In [ ]:
np.array([sum(count_2[0]),sum(count_2[1]),sum(count_2[2]),sum(count_2[3])])

In [ ]:
#importing control data
control = pd.read_excel ('/Users/liu/Desktop/合作社專案三_lungcancer/data(new)/LoD+LoB(new heater) data整理-03102020.xlsx',sheet_name=6)

In [ ]:
#clean the control data
#compare whether the label are the same
control = control.drop(index=[0,1,2]).reset_index(drop=True)
control.iloc[:,0] == merge.iloc[:,0]

In [ ]:
#check the null value counts
control.iloc[:,1:169] = control.iloc[:,1:169].astype('float')
control.isnull().sum(axis=1)

In [ ]:
#level
level = []
for i in range(168):
    if i < 21 or 84 <= i < 105:
        level.append(4)
    if 21 <= i < 42 or 105 <= i < 126:
        level.append(3)
    if 42 <= i < 63 or 126 <= i < 147:
        level.append(2)
    if 63 <= i < 84 or 147 <= i < 168:
        level.append(1)

In [ ]:
#filling the missing data
def fill(row):
    mean = np.mean(row[1:])
    row = row.fillna(mean)
    return row
control = control.apply(fill,axis=1)

In [ ]:
#for each mutation
#draw a plot that have category 0 and 1
#jittering
#color them with different concentration

In [ ]:
#testing the code for plotting a mutation point
mutation_c = pd.DataFrame()
mutation_c['response'] = control.iloc[0,1:]
mutation_c['mutation'] = 0
mutation_c['level'] = 0
mutation_m = pd.DataFrame()
mutation_m['response'] = merge.iloc[0,1:]
mutation_m['mutation'] = 1
mutation_m = pd.concat([mutation_m.reset_index(drop=True),pd.DataFrame(level,columns=['level'])],axis=1)
mutation = pd.concat([mutation_c.reset_index(drop=True),mutation_m],axis=0).reset_index(drop=True)
mutation['response'] = mutation['response'].astype('int')

In [ ]:
sns.scatterplot(data=mutation,x='response',y='mutation',hue='level',alpha=0.6)

In [ ]:
plt.rcParams["font.family"] = "Times"
ax = sns.boxplot(data=mutation, x='mutation', y='response',hue='level')

In [ ]:
#regplot seems to be slow
sns.regplot(x=mutation['response']+1,y=mutation['mutation'],logistic=True)

In [ ]:
###next steps
#sum up what we've done here
#jitterplot ###can't find efficient ways:P###
#learn to write out the graph
#logistic regression
#data preprocessing
#sample mutation for visualization
#generalize to all mutation
#better way to run for different data?
#discuss the conclusion

In [ ]:
#fit the data with sklearn logistic
#and plot the regression line with the scatterplot
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
X = mutation['response'].values.reshape(-1,1)
Y = mutation['mutation'].values.reshape(-1,1)
clf.fit(X,np.ravel(Y.astype(int)))

In [ ]:
curve = np.linspace(0,max(mutation['response']),100).reshape(-1,1)
plt.plot(curve,clf.predict_proba(curve)[:,1],alpha=0.7,color='lightblue')
sns.scatterplot(data=mutation,x='response',y='mutation',hue='level',alpha=0.6)
plt.show()

In [ ]:
###randomly sample 7 mutation points to be visiualized
import random
random.seed(49)
demo_ind = random.sample(range(49),7)

In [ ]:
clf = LogisticRegression()
X = mutation['response'].values.reshape(-1,1)
Y = mutation['mutation'].values.reshape(-1,1)
clf.fit(X,np.ravel(Y.astype(int)))
from sklearn.metrics import accuracy_score
accuracy_score(Y,clf.predict(X))

In [ ]:
#find the upper and lower bound of the gray area
#set max(natural) as upper bound
#lower bound 1.proba50 2.roc curve tp/p+tn/n 3.accuracy

In [ ]:
#return the responsed value corresponding to 50%cutoff
def proba_50(clf):
    cutoff_proba50 = -(clf.intercept_/clf.coef_)
    return cutoff_proba50

In [ ]:
#draw out the box plot of five levels
#the scatter plot and logistic regression
#the proba50 cutoff
import time
start = time.time()
curve = np.linspace(0,max(mutation['response']),100).reshape(-1,1)
for i in range(49):
    mutation_c = pd.DataFrame()
    mutation_c['response'] = control.iloc[i,1:]
    mutation_c['mutation'] = 0
    mutation_c['level'] = 0
    mutation_m = pd.DataFrame()
    mutation_m['response'] = merge.iloc[i,1:]
    mutation_m['mutation'] = 1
    mutation_m = pd.concat([mutation_m.reset_index(drop=True),pd.DataFrame(level,columns=['level'])],axis=1)
    mutation = pd.concat([mutation_c.reset_index(drop=True),mutation_m],axis=0).reset_index(drop=True)
    mutation['response'] = mutation['response'].astype('int')
    clf = LogisticRegression()
    X = mutation['response'].values.reshape(-1,1)
    Y = mutation['mutation'].values.reshape(-1,1)
    print('plot of mutation:',i,'-',merge.iloc[i,0])
    clf.fit(X,np.ravel(Y.astype(int)))
    ax = sns.boxplot(data=mutation, x='mutation', y='response',hue='level')
    plt.show()
    sns.scatterplot(data=mutation,x='response',y='mutation',hue='level',alpha=0.6)
    plt.plot(curve,clf.predict_proba(curve)[:,1],alpha=0.7,color='lightblue')
    plt.show()
    print('the cutoff point for',i,'-',merge.iloc[i,0], proba_50(clf))
end = time.time()
print('running time: ',end-start)

In [ ]:
#count the false outside gray area
def count_large_native(data,upperbound):
    count = sum(((data['mutation']==0)&(data['response']>upperbound)))
    return count
def count_small_mutation(data,lowerbound):
    count = sum(((data['mutation']==1)&(data['response']<lowerbound)))
    return count

In [ ]:
#count the number in gray area
def count_num(data,lower,upper):
    count = 0
    for i in data:
        if lower <= i <= upper:
            count += 1
    return count

In [ ]:
#method1
#upper bound = max(native)
#lower bound = proba50 cutoff
start = time.time()
method_1 = pd.DataFrame(columns=['Index','Name','Lower','Upper','Gray rate','FN rate','Native','Mutated','Bignative','Smallmutation'])
for i in range(49):
    #build dataframe for each mutation
    #control
    mutation_c = pd.DataFrame()
    mutation_c['response'] = control.iloc[i,1:]
    mutation_c['mutation'] = 0
    mutation_c['level'] = 0
    #mutated
    mutation_m = pd.DataFrame()
    mutation_m['response'] = merge.iloc[i,1:]
    mutation_m['mutation'] = 1
    mutation_m = pd.concat([mutation_m.reset_index(drop=True),pd.DataFrame(level,columns=['level'])],axis=1)
    mutation = pd.concat([mutation_c.reset_index(drop=True),mutation_m],axis=0).reset_index(drop=True)
    mutation['response'] = mutation['response'].astype('int')
    #fit
    clf = LogisticRegression()
    X = mutation['response'].values.reshape(-1,1)
    Y = mutation['mutation'].values.reshape(-1,1)
    clf.fit(X,np.ravel(Y.astype(int)))
    #calculating lower and upper bounds
    upper = max(mutation_c.iloc[:,0])
    lower = float(proba_50(clf))        
    if lower > upper:
        print('the bounds of gray area of ','***',i,merge.iloc[i,0],'***',' is ')
        print('no gray area')
        #store np.nan
        method_1 = method_1.append({'Index':i,'Name':merge.iloc[i,0],'Lower':lower,'Upper':upper,'Gray rate':np.nan,'FN rate':np.nan,'Native':np.nan,'Mutated':np.nan,'Bignative':np.nan,'Smallmutation':np.nan},ignore_index=True)
    else:
    #count the proportion of data in the gray area
        gray_num = count_num(mutation.iloc[:,0],lower,upper)
        native = count_num(mutation_c.iloc[:,0],lower,upper)
        mutated = count_num(mutation_m.iloc[:,0],lower,upper)
        big_native = count_large_native(mutation,upper)
        small_mutation = count_small_mutation(mutation,lower)
    #store into a dataframe
        method_1 = method_1.append({'Index':i,'Name':merge.iloc[i,0],'Lower':lower,'Upper':upper,'Gray rate':gray_num/len(mutation),'FN rate':mutated/(native+mutated),'Native':native,'Mutated':mutated,'Bignative':big_native,'Smallmutation':small_mutation},ignore_index=True)
    #print out the result
        print('the bounds of gray area of ','***',i,merge.iloc[i,0],'***',' is ','( {} , {} )'.format(lower,upper))
        print('the proportion of data in the gray area is ',gray_num/len(mutation))
        print('There are {} native and {} mutated in gray area: False Negative {}%'.format(native,mutated,mutated/(native+mutated)))
        print('***number of large_native is {}, number of small_mutation is {}'.format(big_native,small_mutation))
#out put the result as csv
method_1.to_csv('method_1.csv')
#how long does it take?
end = time.time()
print('running time: ',end-start)

In [ ]:
def cut_data_1000(data):
    cut_low = min(data)
    cut_high = max(data)
    cut = np.linspace(cut_low,cut_high,1000)
    return(cut)

def find_roc_best(mutation_c,mutation_m,cut):
    SUM_best = 0
    best_cut = 0
    for i in cut:
        TN_N = sum(mutation_c.iloc[:,0] < i) / len(mutation_c)
        TP_P = sum(mutation_m.iloc[:,0] > i) / len(mutation_m)
        SUM = TN_N+TP_P
        if SUM > SUM_best:
            best_cut = i
            SUM_best = SUM
    return(best_cut,SUM_best)
        
def find_max_accu(mutation_c,mutation_m,mutation,cut):
    accuracy_best = 0
    best_cut = 0
    for i in cut:
        TN = sum(mutation_c.iloc[:,0] < i)
        TP = sum(mutation_m.iloc[:,0] > i)
        accuracy = (TP+TN)/len(mutation)
        if accuracy > accuracy_best:
            best_cut = i
            accuracy_best = accuracy
    return(best_cut,accuracy_best)

In [ ]:
#upper bound = max(native)
#lower bound = best roc cutoff
start = time.time()
method_2 = pd.DataFrame(columns=['Index','Name','Lower','Upper','Gray rate','FN rate','Native','Mutated','Bignative','Smallmutation'])
for i in range(49):
    #build dataframe for each mutation
    #control
    mutation_c = pd.DataFrame()
    mutation_c['response'] = control.iloc[i,1:]
    mutation_c['mutation'] = 0
    mutation_c['level'] = 0
    #mutated
    mutation_m = pd.DataFrame()
    mutation_m['response'] = merge.iloc[i,1:]
    mutation_m['mutation'] = 1
    mutation_m = pd.concat([mutation_m.reset_index(drop=True),pd.DataFrame(level,columns=['level'])],axis=1)
    mutation = pd.concat([mutation_c.reset_index(drop=True),mutation_m],axis=0).reset_index(drop=True)
    mutation['response'] = mutation['response'].astype('int')
    #calculating lower and upper bounds
    upper = max(mutation_c.iloc[:,0])
    #cut the data
    cut = cut_data_1000(mutation.iloc[:,0])
    #find the best criteria that separate Postive and Negative
    lower,SUM_best = find_roc_best(mutation_c,mutation_m,cut)
    if lower > upper:
        print('the bounds of gray area of ','***',i,merge.iloc[i,0],'***',' is ')
        print('no gray area')
        #store np.nan
        method_2 = method_2.append({'Index':i,'Name':merge.iloc[i,0],'Lower':lower,'Upper':upper,
                                    'Gray rate':np.nan,'FN rate':np.nan,'Native':np.nan,'Mutated':np.nan,
                                    'Bignative':np.nan,'Smallmutation':np.nan},ignore_index=True)
    else:
    #count the proportion of data in the gray area
        gray_num = count_num(mutation.iloc[:,0],lower,upper)
        native = count_num(mutation_c.iloc[:,0],lower,upper)
        mutated = count_num(mutation_m.iloc[:,0],lower,upper)
        big_native = count_large_native(mutation,upper)
        small_mutation = count_small_mutation(mutation,lower)
        #store into a dataframe
        method_2 = method_2.append({'Index':i,'Name':merge.iloc[i,0],'Lower':lower,'Upper':upper,
                                    'Gray rate':gray_num/len(mutation),'FN rate':mutated/(native+mutated),
                                    'Native':native,'Mutated':mutated,'Bignative':big_native,'Smallmutation':small_mutation},
                                   ignore_index=True)
    #print out the result
        print('the Best SUM of the cut :',SUM_best)
        print('the bounds of gray area of ','***',i,merge.iloc[i,0],'***',' is ','( {} , {} )'.format(lower,upper))
        print('the proportion of data in the gray area is ',gray_num/len(mutation))
        print('There are {} native and {} mutated in gray area: False Negative {}%'.format(native,mutated,mutated/
                                                                                           (native+mutated)))
        print('***number of large_native is {}, number of small_mutation is {}'.format(big_native,
                                                                                       small_mutation))
        #output the result as csv
method_2.to_csv('method_2.csv')
#how long does it take?
end = time.time()
print('running time: ',end-start)

In [ ]:
#upper bound = max(native)
#lower bound = best accuracy
start = time.time()
method_3 = pd.DataFrame(columns=['Index','Name','Lower','Upper','Gray rate','FN rate','Native','Mutated','Bignative','Smallmutation'])
for i in range(49):
    #build dataframe for each mutation
    #control
    mutation_c = pd.DataFrame()
    mutation_c['response'] = control.iloc[i,1:]
    mutation_c['mutation'] = 0
    mutation_c['level'] = 0
    #mutated
    mutation_m = pd.DataFrame()
    mutation_m['response'] = merge.iloc[i,1:]
    mutation_m['mutation'] = 1
    mutation_m = pd.concat([mutation_m.reset_index(drop=True),pd.DataFrame(level,columns=['level'])],axis=1)
    mutation = pd.concat([mutation_c.reset_index(drop=True),mutation_m],axis=0).reset_index(drop=True)
    mutation['response'] = mutation['response'].astype('int')
    #calculating lower and upper bounds
    upper = max(mutation_c.iloc[:,0])
    #cut the data
    cut = cut_data_1000(mutation.iloc[:,0])
    #find the best criteria that separate Postive and Negative
    lower,accuracy_best = find_max_accu(mutation_c,mutation_m,mutation,cut)
    if lower > upper:
        print('the bounds of gray area of ','***',i,merge.iloc[i,0],'***',' is ')
        print('no gray area')
    #store np.nan
        method_3 = method_3.append({'Index':i,'Name':merge.iloc[i,0],'Lower':lower,'Upper':upper,
                                    'Gray rate':np.nan,'FN rate':np.nan,'Native':np.nan,'Mutated':np.nan,
                                    'Bignative':np.nan,'Smallmutation':np.nan},ignore_index=True)
    else:
    #count the proportion of data in the gray area
        gray_num = count_num(mutation.iloc[:,0],lower,upper)
        native = count_num(mutation_c.iloc[:,0],lower,upper)
        mutated = count_num(mutation_m.iloc[:,0],lower,upper)
        big_native = count_large_native(mutation,upper)
        small_mutation = count_small_mutation(mutation,lower)
        #store into a dataframe
        method_3 = method_3.append({'Index':i,'Name':merge.iloc[i,0],'Lower':lower,'Upper':upper,
                                    'Gray rate':gray_num/len(mutation),'FN rate':mutated/(native+mutated),
                                    'Native':native,'Mutated':mutated,'Bignative':big_native,'Smallmutation':small_mutation},
                                   ignore_index=True)
    #print out the result
        print('the Best SUM of the cut :',SUM_best)
        print('the bounds of gray area of ','***',i,merge.iloc[i,0],'***',' is ','( {} , {} )'.format(lower,upper))
        print('the proportion of data in the gray area is ',gray_num/len(mutation))
        print('There are {} native and {} mutated in gray area: False Negative {}%'.format(native,mutated,mutated/
                                                                                           (native+mutated)))
        print('***number of large_native is {}, number of small_mutation is {}'.format(big_native,
                                                                                       small_mutation))
#output the result as csv
method_3.to_csv('method_3.csv')
#how long does it take?
end = time.time()
print('running time: ',end-start)

In [ ]:
#define the bootstrap method and it returns the list of all proba_50 values
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
def bootstrap(data,sam_pro,repeat_times):
    bootstrap_proba50 = []
    for i in range(repeat_times):
        sample = data.sample(frac=sam_pro)
        X = sample['response'].values.reshape(-1,1)+0.1
        Y = sample['mutation'].values.reshape(-1,1)
        clf.fit(X,np.ravel(Y.astype(int)))
        proba50 = float(-(clf.intercept_/clf.coef_))
        bootstrap_proba50.append(proba50)
    return bootstrap_proba50

In [ ]:
#testing the function for a mutation
bootstrap_proba50 = bootstrap(mutation,0.8,1000)
print('the upper bound is ',max(bootstrap_proba50),', the lower bound is ',min(bootstrap_proba50))

In [ ]:
merge.iloc[:,1:].astype(float).transpose().describe().to_csv('merge_summary.csv')

In [ ]:
control.iloc[:,1:].astype(float).transpose().describe().to_csv('control_summary.csv')

In [ ]:
#fourth method
#conduct bootstrap method to find the distribution proba50
start = time.time()
method_4 = pd.DataFrame(columns=['Index','Name','Lower','Upper','Gray rate','FN rate','Native','Mutated','Bignative','Smallmutation'])
proba50_dis = []
for i in range(49):
    #build dataframe for each mutation
    #control
    mutation_c = pd.DataFrame()
    mutation_c['response'] = control.iloc[i,1:]
    mutation_c['mutation'] = 0
    mutation_c['level'] = 0
    #mutated
    mutation_m = pd.DataFrame()
    mutation_m['response'] = merge.iloc[i,1:]
    mutation_m['mutation'] = 1
    mutation_m = pd.concat([mutation_m.reset_index(drop=True),pd.DataFrame(level,columns=['level'])],axis=1)
    mutation = pd.concat([mutation_c.reset_index(drop=True),mutation_m],axis=0).reset_index(drop=True)
    mutation['response'] = mutation['response'].astype('int')
    #bootstrap
    bootstrap_proba50 = bootstrap(mutation,0.8,1000) #this is a list of 1000 proba50
    proba50_dis.append(bootstrap_proba50)
    #calculating lower and upper bounds
    upper = max(bootstrap_proba50)
    lower = min(bootstrap_proba50)
    gray_num = count_num(mutation.iloc[:,0],lower,upper)
    native = count_num(mutation_c.iloc[:,0],lower,upper)
    mutated = count_num(mutation_m.iloc[:,0],lower,upper)
    big_native = count_large_native(mutation,upper)
    small_mutation = count_small_mutation(mutation,lower)
    #store into a df
    try:
        FN_rate = mutated/(native+mutated)
    except:
        FN_rate = 0
    method_4 = method_4.append({'Index':i,'Name':merge.iloc[i,0],'Lower':lower,'Upper':upper,
                                'Gray rate':gray_num/len(mutation),'FN rate':FN_rate,
                                'Native':native,'Mutated':mutated,'Bignative':big_native,'Smallmutation':small_mutation},
                                ignore_index=True)
    #print out the result
    print('the bounds of gray area of ','***',i,merge.iloc[i,0],'***',' is ','( {} , {} )'.format(lower,upper))
    print('the proportion of data in the gray area is ',gray_num/len(mutation))
    print('There are {} native and {} mutated in gray area: False Negative {}%'.format(native,mutated,FN_rate))
    print('***number of large_native is {}, number of small_mutation is {}'.format(big_native,
                                                                                       small_mutation))
#output the result as csv
method_4.to_csv('method_4.csv')
#how long does it take?
end = time.time()
print('running time: ',end-start)

In [ ]:
#last time output4 csv files
#put them into a excel and start to write analysis and discussion
sns.displot(bootstrap_proba50)

In [ ]:
proba50_dis = pd.read_csv('proba50_distribution.csv').drop(columns='Unnamed: 0')

In [ ]:
for i in range(49):
    sns.displot(proba50_dis.iloc[i,:])

In [ ]:
#check #將比較標準定義在gray area外
#check #討論bootstrap分佈異常 #次數增大出現 多峰常態的現象 可以以26舉例來討論分析結果
#check #討論批次的異常是否集中在某處 #主要是有極端異常值 或是實驗組之變異量較大


#漏網之魚還是不少 #下界差異較大 #分佈左偏 #討論bootstrap信心水準
#視覺化gray area


#程式碼已寫好，估計要跑八小時 #將法一法二法三都做bootstrap

In [ ]:
#test the suitable bootstrap times for mutation26
#choose it because of its variation

import time
start = time.time()
proba50_dis = []
#build dataframe for each mutation
#control
mutation_c = pd.DataFrame()
mutation_c['response'] = control.iloc[26,1:]
mutation_c['mutation'] = 0
mutation_c['level'] = 0
#mutated
mutation_m = pd.DataFrame()
mutation_m['response'] = merge.iloc[26,1:]
mutation_m['mutation'] = 1
mutation_m = pd.concat([mutation_m.reset_index(drop=True),pd.DataFrame(level,columns=['level'])],axis=1)
mutation = pd.concat([mutation_c.reset_index(drop=True),mutation_m],axis=0).reset_index(drop=True)
mutation['response'] = mutation['response'].astype('int')
#bootstrap
boot_test = [1000,2000,3000,4000,5000,10000]
for i in boot_test:
    bootstrap_proba50 = bootstrap(mutation,0.8,i) #this is a list of 1000 proba50
    print('iteration',i,'times')
    sns.displot(bootstrap_proba50)
    plt.show()
print('running time: ',end-start)

In [ ]:
#method2的bootstrap版本，但使用大量迴圈，所需執行時間長，1000次重抽樣、1000cut，每一突變點需約10分鐘
def bootstrap_roc(data,sam_pro,repeat_times,cut_times):
    bootstrap_roc = []
    for i in range(repeat_times):
        sample = data.sample(frac=sam_pro)
        cut = np.linspace(min(sample['response']),max(sample['response']),cut_times)
        best_cut,SUM_best = find_roc_best(sample[sample.mutation==0],sample[sample.mutation==1],cut)
        bootstrap_roc.append(best_cut)
    return bootstrap_roc

In [ ]:
#method3的bootstrap版本，耗時長、暫不考慮
def bootstrap_acc(data,sam_pro,repeat_times,cut_times):
    bootstrap_acc = []
    for i in range(repeat_times):
        sample = data.sample(frac=sam_pro)
        cut = np.linspace(min(sample['response']),max(sample['response']),cut_times)
        best_cut,accuracy_best = find_max_accu(sample[sample.mutation==0],sample[sample.mutation==1],cut)
        bootstrap_acc.append(best_cut)
    return bootstrap_acc

In [ ]:
#fifth method
#conduct bootstrap method to find the distribution best roc_cut
start = time.time()
method_5 = pd.DataFrame(columns=['Index','Name','Lower','Upper','Gray rate','FN rate','Native','Mutated','Bignative','Smallmutation'])
roc_dis = []
for i in range(49):
    #build dataframe for each mutation
    #control
    mutation_c = pd.DataFrame()
    mutation_c['response'] = control.iloc[i,1:]
    mutation_c['mutation'] = 0
    mutation_c['level'] = 0
    #mutated
    mutation_m = pd.DataFrame()
    mutation_m['response'] = merge.iloc[i,1:]
    mutation_m['mutation'] = 1
    mutation_m = pd.concat([mutation_m.reset_index(drop=True),pd.DataFrame(level,columns=['level'])],axis=1)
    mutation = pd.concat([mutation_c.reset_index(drop=True),mutation_m],axis=0).reset_index(drop=True)
    mutation['response'] = mutation['response'].astype('int')
    #bootstrap
    bootstrap_roc = bootstrap_roc(mutation,0.8,1000,2000)
    roc_dis.append(bootstrap_roc)
    #calculating lower and upper bounds
    upper = max(bootstrap_roc)
    lower = min(bootstrap_roc)
    gray_num = count_num(mutation.iloc[:,0],lower,upper)
    native = count_num(mutation_c.iloc[:,0],lower,upper)
    mutated = count_num(mutation_m.iloc[:,0],lower,upper)
    big_native = count_large_native(mutation,upper)
    small_mutation = count_small_mutation(mutation,lower)
    #store into a df
    try:
        FN_rate = mutated/(native+mutated)
    except:
        FN_rate = 0
    method_5 = method_5.append({'Index':i,'Name':merge.iloc[i,0],'Lower':lower,'Upper':upper,
                                'Gray rate':gray_num/len(mutation),'FN rate':FN_rate,
                                'Native':native,'Mutated':mutated,'Bignative':big_native,'Smallmutation':small_mutation},
                                ignore_index=True)
    #print out the result
    print('the bounds of gray area of ','***',i,merge.iloc[i,0],'***',' is ','( {} , {} )'.format(lower,upper))
    print('the proportion of data in the gray area is ',gray_num/len(mutation))
    print('There are {} native and {} mutated in gray area: False Negative {}%'.format(native,mutated,FN_rate))
    print('***number of large_native is {}, number of small_mutation is {}'.format(big_native,
                                                                                       small_mutation))
#output the result as csv
method_5.to_csv('method_5.csv')
#how long does it take?
end = time.time()
print('running time: ',end-start)

In [ ]:
#fourth method #Second edition
#conduct bootstrap method to find the distribution proba50
#use the 5 and 95 percentile as lower and upper
start = time.time()
method_4_2 = pd.DataFrame(columns=['Index','Name','Lower','Upper','Gray rate','FN rate','Native','Mutated','Bignative','Smallmutation'])
proba50_dis = []
for i in range(49):
    #build dataframe for each mutation
    #control
    mutation_c = pd.DataFrame()
    mutation_c['response'] = control.iloc[i,1:]
    mutation_c['mutation'] = 0
    mutation_c['level'] = 0
    #mutated
    mutation_m = pd.DataFrame()
    mutation_m['response'] = merge.iloc[i,1:]
    mutation_m['mutation'] = 1
    mutation_m = pd.concat([mutation_m.reset_index(drop=True),pd.DataFrame(level,columns=['level'])],axis=1)
    mutation = pd.concat([mutation_c.reset_index(drop=True),mutation_m],axis=0).reset_index(drop=True)
    mutation['response'] = mutation['response'].astype('int')
    #bootstrap
    bootstrap_proba50 = bootstrap(mutation,0.8,1000) #this is a list of 1000 proba50
    proba50_dis.append(bootstrap_proba50)
    #calculating lower and upper bounds
    upper = np.percentile(bootstrap_proba50,95)
    lower = np.percentile(bootstrap_proba50,5)
    gray_num = count_num(mutation.iloc[:,0],lower,upper)
    native = count_num(mutation_c.iloc[:,0],lower,upper)
    mutated = count_num(mutation_m.iloc[:,0],lower,upper)
    big_native = count_large_native(mutation,upper)
    small_mutation = count_small_mutation(mutation,lower)
    #store into a df
    try:
        FN_rate = mutated/(native+mutated)
    except:
        FN_rate = 0
    method_4_2 = method_4_2.append({'Index':i,'Name':merge.iloc[i,0],'Lower':lower,'Upper':upper,
                                'Gray rate':gray_num/len(mutation),'FN rate':FN_rate,
                                'Native':native,'Mutated':mutated,'Bignative':big_native,'Smallmutation':small_mutation},
                                ignore_index=True)
    #print out the result
    print('the bounds of gray area of ','***',i,merge.iloc[i,0],'***',' is ','( {} , {} )'.format(lower,upper))
    print('the proportion of data in the gray area is ',gray_num/len(mutation))
    print('There are {} native and {} mutated in gray area: False Negative {}%'.format(native,mutated,FN_rate))
    print('***number of large_native is {}, number of small_mutation is {}'.format(big_native,
                                                                                       small_mutation))
#output the result as csv
method_4_2.to_csv('method_4_2.csv')
#how long does it take?
end = time.time()
print('running time: ',end-start)

In [ ]:
#visualize all 5 methods on the following index(reason listed in the excel file)
vis_ind = [0,9,12,31,43,44]
import time
start = time.time()
boundaries = pd.read_excel('/Users/liu/Desktop/合作社專案三_lungcancer/資料彙整.xlsx',sheet_name='visualized mutation')
plt.rcParams["figure.figsize"] = (20,10)
for i,j in enumerate(vis_ind):
    mutation_c = pd.DataFrame()
    mutation_c['response'] = control.iloc[j,1:]
    mutation_c['mutation'] = 0
    mutation_c['level'] = 0
    mutation_m = pd.DataFrame()
    mutation_m['response'] = merge.iloc[j,1:]
    mutation_m['mutation'] = 1
    mutation_m = pd.concat([mutation_m.reset_index(drop=True),pd.DataFrame(level,columns=['level'])],axis=1)
    mutation = pd.concat([mutation_c.reset_index(drop=True),mutation_m],axis=0).reset_index(drop=True)
    mutation['response'] = mutation['response'].astype('int')
    clf = LogisticRegression()
    X = mutation['response'].values.reshape(-1,1)
    Y = mutation['mutation'].values.reshape(-1,1)
    clf.fit(X,np.ravel(Y.astype(int)))
    print('plot of mutation:',j,'-',merge.iloc[j,0])
    for k in range(1,6):
        lower = float(boundaries.iloc[i+1,k+1])
        upper = float(boundaries.iloc[i+1,k+6])
        plt.subplot(2, 3, k)
        sns.scatterplot(data=mutation,x='response',y='mutation',hue='level',alpha=0.6).set_title(label=boundaries.iloc[0,k+1])
        plt.plot(curve,clf.predict_proba(curve)[:,1],alpha=0.7,color='lightblue')
        plt.axvspan(lower, upper, facecolor='gray', alpha=0.3)
        
    plt.show()
end = time.time()
print('running time: ',end-start)